The Clusters dataset is aimed at classifiying 6 different colours and shapes in a picture.

Importing the files from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
mkdir clusters_two_data

In [ ]:
ls

clusters_two_data/  drive/  sample_data/


In [ ]:
cd clusters_two_data

/content/clusters_two_data


In [ ]:
ls

In [ ]:
!cp /content/drive/'My Drive'/'Colab Notebooks'/'Image Classification'/clusters_two_categories.zip .

In [ ]:
ls

clusters_two_categories.zip


In [ ]:
# Extracting the files
import zipfile
path_to_zip_file = 'clusters_two_categories.zip'
directory_to_extract_to =''
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [ ]:
### remove the zip file
!rm clusters_two_categories.zip

In [ ]:
ls

clusters_two_categories/


In [ ]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np

In [ ]:
# load in the training data set

train_df = pd.read_csv('/content/clusters_two_data/clusters_two_categories/data/train.csv')

In [ ]:
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^


In [ ]:
train_df['color'].unique()

array(['red', 'blue', 'green', 'teal', 'orange', 'purple'], dtype=object)

In [ ]:
train_df['marker'].unique()

array(['^', '+', '*'], dtype=object)

In [ ]:
# the next is to use the pd.get_dummies to categorize the data using one hot encoder

one_hot_color = pd.get_dummies(train_df.color).values
one_hot_marker = pd.get_dummies(train_df.marker).values

# contactenate the two numpy arrays above

labels = np.concatenate((one_hot_color, one_hot_marker), axis=1)


# Build a neural network around this training data.
# first thing is to pass a keras sequential type. 
# The sequetial allows us list the different layers we have in our neural network.
# this line helps us access layers. the input shape is 2 cos its just x and y and the output shape is also 2
# the 4 is the number of neurons (nodes) in the hidden layers 


model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(2,), activation='relu'), 
    #keras.layers.Dropout(0.2), #this drops 20% of the nodes in the hidden layer
    keras.layers.Dense(64, activation='relu'), #this adds another hidden layers
    keras.layers.Dense(9, activation='sigmoid')])


# next is to compile our model i.e to setup loss for our network. this is going to tell us how to train.

model.compile(optimizer='adam',
             loss=keras.losses.BinaryCrossentropy(from_logits=True), #this updates the network
             metrics=['accuracy'])

# next is to fit the training data into our network

#model.fit([x], train_df.color.values, batch_size = 16)
#Since [x] is a combination of x and y in the train_df, we have to stack both (pair) columns together using the below code

x = np.column_stack((train_df.x.values, train_df.y.values))

# shuffle in an order using the code below

np.random.RandomState(seed=42).shuffle(x)
np.random.RandomState(seed=42).shuffle(labels)

model.fit([x], labels, batch_size = 16, epochs=10)

# # Evaluate on the test data

test_df = pd.read_csv('/content/clusters_two_data/clusters_two_categories/data/test.csv')
test_x = np.column_stack((test_df.x.values, test_df.y.values))

test_one_hot_color = pd.get_dummies(test_df.color).values
test_one_hot_marker = pd.get_dummies(test_df.marker).values

# contactenate the two numpy arrays above

test_labels = np.concatenate((test_one_hot_color, test_one_hot_marker), axis=1)

#  to evaluate use:

print('EVALUATION')

model.evaluate(test_x, test_labels)

print('Prediction =', np.round(model.predict(np.array([[0, 3], [0,1], [-2, 1]]))))

Epoch 1/10
375/375 [==============================] - 1s 1ms/step - loss: 0.4303 - accuracy: 0.0674
Epoch 2/10
375/375 [==============================] - 0s 1ms/step - loss: 0.1621 - accuracy: 0.2604
Epoch 3/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0954 - accuracy: 0.2938
Epoch 4/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0663 - accuracy: 0.2719
Epoch 5/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0509 - accuracy: 0.2564
Epoch 6/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0445 - accuracy: 0.2615
Epoch 7/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0392 - accuracy: 0.2637
Epoch 8/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0330 - accuracy: 0.2613
Epoch 9/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0312 - accuracy: 0.2708
Epoch 10/10
375/375 [==============================] - 0s 1ms/step - loss: 0.0299 - accuracy: 0.2647